# t|ket> benchmarking example


The aim of this example is to show how to run the IBM benchmarking circuits through t|ket>. The benchmarking circuits originated from https://github.com/iic-jku/ibm_qx_mapping/tree/master/examples, but there is a copy in pytket in the "benchmarking" folder. The initial circuits are written in QASM, meaning that they must be converted to t|ket>'s internal representation via Qiskit. Using this script we will compile these circuits through t|ket> and then print a table to terminal containing analysis of the circuits post-compilation.

First, begin by importing qiskit and pytket.

In [1]:
import qiskit
import pytket

We need access to Qiskit's `QuantumCircuit` and the `circuit_to_dag` converter, which will allow us to take the original QASM and retrieve a Qiskit DAGCircuit. There is then a pytket method `dagcircuit_to_tk` to convert this DAGCircuit to t|ket>'s internal representation. We also need the pytket method `coupling_to_arc` to convert IBM's architectures to directed graphs for the use of routing.

In [2]:
from qiskit import QuantumCircuit
from qiskit.converters import circuit_to_dag

from pytket.qiskit import dagcircuit_to_tk, coupling_to_arc

We need all of the required pytket equipment to allow us to perform clean-up transform passes, to route and to analyse the circuits. Lastly, we require the `pandas` module to hold our data, and 
`time` to benchmark compilation time per circuit.

In [3]:
from pytket._routing import Architecture, route
from pytket._circuit import OpType, optimise_pre_routing, optimise_post_routing
import pandas, time

The architectures used to benchmark for our routing paper were the IBMqx5 and IBMTokyo architectures. The architectures qx5 and Tokyo respectively are shown below (although the diagrams do not show the directedness of the coupling maps). These diagrams are from https://www.research.ibm.com/ibm-q/technology/devices/#ibmqx5. We will now define the coupling map representing both architectures. These will later be converted to directed graphs to be used by t|ket>.
![alt text](IBMqx5Arc.png "Title")
![alt text](IBMTokyoArc.png "Title2")

In [4]:
ibm_devices = {
    "ibmqx5": {"edges": [(1, 0), (1, 2), (2, 3), (3, 4), 
    (3, 14), (5, 4), (6, 5), (6, 7), (6, 11), (7, 10), 
    (8, 7), (9, 8), (9, 10), (11, 10), (12, 5), (12, 11), 
    (12, 13), (13, 4), (13, 14), (15, 0), (15, 2), (15, 14)],
    "nodes": 16},
    "ibmq_20_tokyo": {"edges": [(0, 1), (0, 5), (1, 2), (1, 6), (1, 7), (2, 3), (2, 6),
    (2, 7), (3, 4), (3, 8), (3, 9), (4, 8), (4, 9), (5, 6), (5, 10), (5, 11), (6, 7),
    (6, 10), (6, 11), (7, 8), (7, 12), (7, 13), (8, 9), (8, 12), (8, 13), (9, 14), (10, 11),
    (10, 15), (11, 12), (11, 16), (11, 17), (12, 13), (12, 16), (12, 17), (13, 14), (13, 18),
    (13, 19), (14, 18), (14, 19), (15, 16), (16, 17), (17, 18), (18, 19)], "nodes": 20}
}

We shall now choose the device and create a directed graph for t|ket>'s routing to use.

In [5]:
device_name = 'ibmqx5' #'ibm_20_tokyo' ###Note: can also be ran using the Tokyo machine architecture
                                       ###      or with a user-defined coupling map
coupling_map = ibm_devices[device_name]["edges"]
directed_arc = coupling_to_arc(coupling_map)

We now define a method which takes in (1) a QASM file and (2) a directed graph architecture; it returns analysis of the circuit after our transform passes and routing procedure have been completed. It will also print out to terminal the time taken for all the transformations and routing to finish.

In [6]:
def getStats(filename, directed_arc):
    qc = QuantumCircuit.from_qasm_file(filename)
    dag = circuit_to_dag(qc)
    tkcirc = dagcircuit_to_tk(dag)
    
    start_time = time.process_time()
    optimise_pre_routing(tkcirc)
    outcirc, _ = route(tkcirc, directed_arc)
    optimise_post_routing(outcirc)
    time_elapsed = time.process_time() - start_time
    
    print("Compilation time for circuit " + str(filename) + ": " + str(time_elapsed) + "s")
    if outcirc.n_vertices()==0:
        return [0,0,0,0,0]
    ###Returns: [number of vertices, circuit depth, nubmer of CX gates, number of parallel slices of CX gates]
    return [outcirc.n_vertices()-2*outcirc.n_qubits(), outcirc.depth()-2, outcirc.n_vertices_of_type(OpType.CX), 
            outcirc.depth_by_type(OpType.CX), time_elapsed]
    ###Note: the raw number of vertices in the circuits and the raw depth 
    ###      need to have the i/o vertices removed for fair comparisons

Lastly, we generate the table of input QASM filenames from a csv file using `pandas` and run the circuits through our compiler. The results are printed to terminal by default.

In [7]:
test_table = pandas.read_csv("benchmarking/IBMQConfig.csv",index_col=0)
test_table = test_table.sort_values(by='Depth in')

stat_table = pandas.DataFrame({})
total_time = 0

for i, (index, row) in enumerate(test_table.iterrows()):
    filename = row['Filename']
    new_stats = getStats(filename, directed_arc)
    total_time += new_stats[4] ###
    print("Time spent compiling so far: " + str(total_time))
    new_table_row = pandas.DataFrame.from_dict({index : new_stats}, 
    orient='index', columns=['Size out', 'Depth out', 'CX count out', 'CX depth out','Runtime'])
    stat_table = stat_table.append(new_table_row)

#stat_table.to_csv("BenchmarkTket.csv") ###Note: uncomment this line to print table to csv.
with pandas.option_context('display.max_rows', None):
    print(stat_table)

Compilation time for circuit benchmarking/ibmq/xor5_254.qasm: 0.006245999999999974s
Time spent compiling so far: 0.006245999999999974
Compilation time for circuit benchmarking/ibmq/graycode6_47.qasm: 0.004846000000000572s
Time spent compiling so far: 0.011092000000000546
Compilation time for circuit benchmarking/ibmq/ex1_226.qasm: 0.00633399999999984s
Time spent compiling so far: 0.017426000000000386
Compilation time for circuit benchmarking/ibmq/4gt11_84.qasm: 0.007312000000000651s
Time spent compiling so far: 0.024738000000001037
Compilation time for circuit benchmarking/ibmq/4mod5-v0_20.qasm: 0.008831999999999951s
Time spent compiling so far: 0.03357000000000099
Compilation time for circuit benchmarking/ibmq/ex-1_166.qasm: 0.008073999999999693s
Time spent compiling so far: 0.04164400000000068
Compilation time for circuit benchmarking/ibmq/4mod5-v1_22.qasm: 0.008600000000000385s
Time spent compiling so far: 0.050244000000001066
Compilation time for circuit benchmarking/ibmq/mod5d1_63

Compilation time for circuit benchmarking/ibmq/rd73_140.qasm: 0.05600199999999944s
Time spent compiling so far: 1.5267800000000022
Compilation time for circuit benchmarking/ibmq/4mod7-v1_96.qasm: 0.03779399999999988s
Time spent compiling so far: 1.5645740000000021
Compilation time for circuit benchmarking/ibmq/4gt4-v0_80.qasm: 0.03887600000000013s
Time spent compiling so far: 1.6034500000000023
Compilation time for circuit benchmarking/ibmq/mod10_176.qasm: 0.03881400000000035s
Time spent compiling so far: 1.6422640000000026
Compilation time for circuit benchmarking/ibmq/0410184_169.qasm: 0.05920599999999965s
Time spent compiling so far: 1.7014700000000023
Compilation time for circuit benchmarking/ibmq/qft_16.qasm: 0.2707999999999995s
Time spent compiling so far: 1.9722700000000017
Compilation time for circuit benchmarking/ibmq/4gt12-v0_88.qasm: 0.04283200000000065s
Time spent compiling so far: 2.0151020000000024
Compilation time for circuit benchmarking/ibmq/rd84_142.qasm: 0.1019339999

Compilation time for circuit benchmarking/ibmq/ham15_107.qasm: 2.3963179999999937s
Time spent compiling so far: 24.202816
Compilation time for circuit benchmarking/ibmq/dc2_222.qasm: 2.7083159999999964s
Time spent compiling so far: 26.911131999999995
Compilation time for circuit benchmarking/ibmq/sqn_258.qasm: 2.8333279999999945s
Time spent compiling so far: 29.74445999999999
Compilation time for circuit benchmarking/ibmq/inc_237.qasm: 3.2687820000000016s
Time spent compiling so far: 33.01324199999999
Compilation time for circuit benchmarking/ibmq/cm85a_209.qasm: 3.3346519999999913s
Time spent compiling so far: 36.34789399999998
Compilation time for circuit benchmarking/ibmq/rd84_253.qasm: 3.9503739999999965s
Time spent compiling so far: 40.29826799999998
Compilation time for circuit benchmarking/ibmq/co14_215.qasm: 5.454491999999988s
Time spent compiling so far: 45.75275999999997
Compilation time for circuit benchmarking/ibmq/root_255.qasm: 5.1612480000000005s
Time spent compiling so 